In [0]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
  
        self.conv1 = nn.Sequential(
            
            nn.Conv2d(1,16,3, bias=False),             #in_channels=1, out_channels=16, kernel_size=(3, 3)
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Dropout(0.1), #26 
            
        )

        self.conv2 = nn.Sequential(
            
            nn.Conv2d(16,16,3, bias=False),            #in_channels=16, out_channels=16, kernel_size=(3, 3)
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Dropout(0.1), #24             
        )

        
        self.conv3 = nn.Sequential(
            
            nn.Conv2d(16,16,3, bias=False),          #in_channels=16, out_channels=16, kernel_size=(3, 3)
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Dropout(0.1), #22 
            
        )

 
        self.conv4 = nn.Sequential(
            nn.MaxPool2d(2, 2),                # output_size = 11 
            nn.Conv2d(16,16,3, bias=False),      #in_channels=16, out_channels=16, kernel_size=(3, 3)
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Dropout(0.1), #9     
        )

        

        self.conv5 = nn.Sequential(   
            nn.Conv2d(16,16,3, bias=False),  #in_channels=16, out_channels=16, kernel_size=(3, 3)
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Dropout(0.1),  #7    
            
        )

        self.conv6 = nn.Sequential(   
            nn.Conv2d(16,16,3, bias=False),   #in_channels=16, out_channels=16, kernel_size=(3, 3)
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Dropout(0.1),  #5  
            
        )

        self.conv7 = nn.Sequential(   
            nn.Conv2d(16,32,3, bias=False),   #in_channels=16, out_channels=32, kernel_size=(3, 3)
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.Dropout(0.1),   #3     
            
        )

        self.fc = nn.Sequential(
            nn.Conv2d(32,10,3)
        )

     
    def forward(self, x):
        
        x = self.conv1(x)   
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.conv5(x)
        x = self.conv6(x)
        x = self.conv7(x)
        x = self.fc(x)
        x = x.view(-1, 10)
        return F.log_softmax(x)

In [5]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 26, 26]             144
              ReLU-2           [-1, 16, 26, 26]               0
       BatchNorm2d-3           [-1, 16, 26, 26]              32
           Dropout-4           [-1, 16, 26, 26]               0
            Conv2d-5           [-1, 16, 24, 24]           2,304
              ReLU-6           [-1, 16, 24, 24]               0
       BatchNorm2d-7           [-1, 16, 24, 24]              32
           Dropout-8           [-1, 16, 24, 24]               0
            Conv2d-9           [-1, 16, 22, 22]           2,304
             ReLU-10           [-1, 16, 22, 22]               0
      BatchNorm2d-11           [-1, 16, 22, 22]              32
          Dropout-12           [-1, 16, 22, 22]               0
        MaxPool2d-13           [-1, 16, 11, 11]               0
           Conv2d-14             [-1, 1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:83: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [6]:
torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)

Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw


Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw


Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw




Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw
Processing...
Done!


In [0]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [8]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 20):
    print("EPOCH:", epoch)
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]

EPOCH: 1


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:83: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0505, Accuracy: 9842/10000 (98.42%)

EPOCH: 2


  0%|          | 0/469 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0216, Accuracy: 9938/10000 (99.38%)

EPOCH: 18


  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0213, Accuracy: 9942/10000 (99.42%)

EPOCH: 19


loss=0.049082282930612564 batch_id=468: 100%|██████████| 469/469 [00:11<00:00, 39.49it/s]



Test set: Average loss: 0.0220, Accuracy: 9940/10000 (99.40%)

